<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Quantum_Machine_Learning_(QML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane

In [ ]:
import pennylane as qml
import torch
import torch.nn as nn

n_qubits = 2
dev = qml.device('default.qubit', wires=n_qubits)

def layer(weights):
    qml.Rot(weights[0, 0], weights[0, 1], weights[0, 2], wires=0)
    qml.Rot(weights[1, 0], weights[1, 1], weights[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])

@qml.qnode(dev, interface='torch')
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    layer(weights)
    return qml.expval(qml.PauliZ(0))

weight_shape = (2, 3)
weights = torch.rand(weight_shape, requires_grad=True)
optimizer = torch.optim.Adam([weights], lr=0.1)

# Example training loop
for step in range(100):
    optimizer.zero_grad()
    inputs = torch.tensor([0.1, 0.2], requires_grad=False)
    loss = (quantum_circuit(inputs, weights) - 0.5) ** 2
    loss.backward()
    optimizer.step()
    if step % 10 == 0:
        print(f"Step {step}, Loss: {loss.item()}")